# Load the Dataset

In [6]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

In [7]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("shanover/disease_symptoms_prec_full")
df = pd.DataFrame(dataset['train'])
df

KeyboardInterrupt: 

# Text Preprocessing

In [ ]:
df = df.rename(columns={'symptoms': 'text', 'disease': 'label'})
df['label']=df['label'].str.lower()
df

,label,text,precautions
0,fungal infection,"itching,skin_rash,nodal_skin_eruptions,dischro...","bath twice, use detol or neem in bathing water..."
1,fungal infection,"skin_rash,nodal_skin_eruptions,dischromic__pat...","bath twice, use detol or neem in bathing water..."
2,fungal infection,"itching,nodal_skin_eruptions,dischromic__patches","bath twice, use detol or neem in bathing water..."
3,fungal infection,"itching,skin_rash,dischromic__patches","bath twice, use detol or neem in bathing water..."
4,fungal infection,"itching,skin_rash,nodal_skin_eruptions","bath twice, use detol or neem in bathing water..."
...,...,...,...
4915,(vertigo) paroymsal positional vertigo,"vomiting,headache,nausea,spinning_movements,lo...","lie down, avoid sudden change in body, avoid a..."
4916,acne,"skin_rash,pus_filled_pimples,blackheads,scurring","bath twice, avoid fatty spicy food, drink plen..."
4917,urinary tract infection,"burning_micturition,bladder_discomfort,foul_sm...","drink plenty of water, increase vitamin c inta..."
4918,psoriasis,"skin_rash,joint_pain,skin_peeling,silver_like_...","wash hands with warm soapy water, stop bleedin..."


In [ ]:
target=df['label'].unique().tolist()
target.sort()
d_map={}
for i, target_var in enumerate(target):
    d_map[i]=target_var

print(d_map)

{0: '(vertigo) paroymsal  positional vertigo', 1: 'acne', 2: 'aids', 3: 'alcoholic hepatitis', 4: 'allergy', 5: 'arthritis', 6: 'bronchial asthma', 7: 'cervical spondylosis', 8: 'chicken pox', 9: 'chronic cholestasis', 10: 'common cold', 11: 'dengue', 12: 'diabetes ', 13: 'dimorphic hemmorhoids(piles)', 14: 'drug reaction', 15: 'fungal infection', 16: 'gastroenteritis', 17: 'gerd', 18: 'heart attack', 19: 'hepatitis a', 20: 'hepatitis b', 21: 'hepatitis c', 22: 'hepatitis d', 23: 'hepatitis e', 24: 'hypertension ', 25: 'hyperthyroidism', 26: 'hypoglycemia', 27: 'hypothyroidism', 28: 'impetigo', 29: 'jaundice', 30: 'malaria', 31: 'migraine', 32: 'osteoarthristis', 33: 'paralysis (brain hemorrhage)', 34: 'peptic ulcer diseae', 35: 'pneumonia', 36: 'psoriasis', 37: 'tuberculosis', 38: 'typhoid', 39: 'urinary tract infection', 40: 'varicose veins'}


In [ ]:
import pandas as pd
df=pd.read_csv('Finaldata.csv')
df=df.drop('precautions',axis=1)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df

,Unnamed: 0,label,text
0,0,13,itching skin rash nodal skin eruptions disc...
1,1,13,skin rash nodal skin eruptions dischromic p...
2,2,13,itching nodal skin eruptions dischromic pat...
3,3,13,itching skin rash dischromic patches
4,4,13,itching skin rash nodal skin eruptions
...,...,...,...
4555,4915,0,vomiting headache nausea spinning movements...
4556,4916,1,skin rash pus filled pimples blackheads scu...
4557,4917,36,burning micturition bladder discomfort foul ...
4558,4918,33,skin rash joint pain skin peeling silver li...


# Getting the model ready

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42,shuffle=True)

In [ ]:
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
import tensorflow as tf
# Loading the dataset to train our model
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=512)

train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [ ]:
tokenized_train_dataset= train_data.map(encode,batched=True)

tokenized_val_dataset= test_data.map(encode,batched=True)

Map:   0%|          | 0/3648 [00:00<?, ? examples/s]

Map:   0%|          | 0/912 [00:00<?, ? examples/s]

# LORA Fine Tuning

In [ ]:
import os
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from peft import LoraConfig, TaskType
lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1)

In [ ]:
from transformers import BertForSequenceClassification
model_checkpoint="roberta-base"
loramodel = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=38)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from peft import get_peft_model
lmodel = get_peft_model(loramodel, lora_config)
lora_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=5,)
trainer = Trainer(
    model=lmodel,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

  0%|          | 0/2280 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 2.551527261734009, 'eval_accuracy': 0.6655701754385965, 'eval_runtime': 85.7633, 'eval_samples_per_second': 10.634, 'eval_steps_per_second': 1.329, 'epoch': 1.0}
{'loss': 3.423, 'grad_norm': 6.31296968460083, 'learning_rate': 3.9035087719298244e-05, 'epoch': 1.1}


  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 0.2707875967025757, 'eval_accuracy': 0.9978070175438597, 'eval_runtime': 81.6629, 'eval_samples_per_second': 11.168, 'eval_steps_per_second': 1.396, 'epoch': 2.0}
{'loss': 0.9033, 'grad_norm': 4.837981700897217, 'learning_rate': 2.8070175438596492e-05, 'epoch': 2.19}


...

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 0.046860627830028534, 'eval_accuracy': 0.9989035087719298, 'eval_runtime': 80.7423, 'eval_samples_per_second': 11.295, 'eval_steps_per_second': 1.412, 'epoch': 4.0}
{'loss': 0.1058, 'grad_norm': 1.289726972579956, 'learning_rate': 6.140350877192982e-06, 'epoch': 4.39}


  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 0.03785937651991844, 'eval_accuracy': 0.9989035087719298, 'eval_runtime': 85.9907, 'eval_samples_per_second': 10.606, 'eval_steps_per_second': 1.326, 'epoch': 5.0}
{'train_runtime': 5233.9483, 'train_samples_per_second': 3.485, 'train_steps_per_second': 0.436, 'train_loss': 1.0310101065719337, 'epoch': 5.0}


TrainOutput(global_step=2280, training_loss=1.0310101065719337, metrics={'train_runtime': 5233.9483, 'train_samples_per_second': 3.485, 'train_steps_per_second': 0.436, 'train_loss': 1.0310101065719337, 'epoch': 5.0})

### Save the model

In [ ]:
saved='roberta'
lmodel.save_pretrained(saved)
lora_tokenizer.save_pretrained(saved)

('roberta/tokenizer_config.json',
 'roberta/special_tokens_map.json',
 'roberta/vocab.json',
 'roberta/merges.txt',
 'roberta/added_tokens.json',
 'roberta/tokenizer.json')

# GUI for the project

In [9]:
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer, RobertaForSequenceClassification

import tensorflow as tf
from tkinter import *
from tkinter import ttk
from tkinter.ttk import *

saved='roberta/'
model_ft=RobertaForSequenceClassification.from_pretrained(saved,num_labels=38)
tokenizer_ft=AutoTokenizer.from_pretrained(saved)


def rawtext(text):
   l=text.split(', ')
   print(l)
   s=''
   for words in l:
        s+=words+'  '
   return s
#Create an instance of Tkinter Frame

win = Tk()
win.configure(background='black')
#Set the geometry
win.geometry("700x250")

# Define a function to return the Input data
def get_data():
   text= entry.get()
   text=rawtext(text)
   print(text)
   d_map={0: '(vertigo) paroymsal  positional vertigo', 1: 'acne', 2: 'aids', 3: 'alcoholic hepatitis', 4: 'arthritis', 5: 'bronchial asthma', 
          6: 'cervical spondylosis', 7: 'chicken pox', 8: 'chronic cholestasis', 9: 'common cold', 10: 'dengue', 11: 'diabetes ', 
          12: 'dimorphic hemmorhoids(piles)',13: 'fungal infection', 14: 'gastroenteritis', 15: 'gerd', 16: 'hepatitis a', 17: 'hepatitis b', 
          18: 'hepatitis c', 19: 'hepatitis d', 20: 'hepatitis e', 21: 'hypertension ', 22: 'hyperthyroidism', 23: 'hypoglycemia', 
          24: 'hypothyroidism', 25: 'impetigo', 26: 'jaundice', 27: 'malaria', 28: 'migraine', 29: 'osteoarthristis', 
          30: 'paralysis (brain hemorrhage)', 31: 'peptic ulcer disease', 32: 'pneumonia', 33: 'psoriasis', 34: 'tuberculosis', 35: 'typhoid', 
          36: 'urinary tract infection', 37: 'varicose veins'}
   
   disease_precautions={'fungal infection': 'bath twice  use detol or neem in bathing water  keep infected area dry  use clean cloths',
                        'gerd': 'avoid fatty spicy food, avoid lying down after eating, maintain healthy weight, exercise',
                        'chronic cholestasis': 'cold baths, anti itch medicine, consult doctor, eat healthy',
                        'peptic ulcer diseae': 'avoid fatty spicy food, consume probiotic food, eliminate milk, limit alcohol',
                        'aids': 'avoid open cuts, wear ppe if possible, consult doctor, follow up',
                        'diabetes ': 'have balanced diet, exercise, consult doctor, follow up',
                        'gastroenteritis': 'stop eating solid food for while, try taking small sips of water, rest, ease back into eating',
                        'bronchial asthma': 'switch to loose cloothing, take deep breaths, get away from trigger, seek help',
                        'hypertension ': 'meditation, salt baths, reduce stress, get proper sleep',
                        'migraine': 'meditation, reduce stress, use poloroid glasses in sun, consult doctor',
                        'cervical spondylosis': 'use heating pad or cold pack, exercise, take otc pain reliver, consult doctor',
                        'paralysis (brain hemorrhage)': 'massage, eat healthy, exercise, consult doctor',
                        'jaundice': 'drink plenty of water, consume milk thistle, eat fruits and high fiberous food, medication',
                        'malaria': 'Consult nearest hospital, avoid oily food, avoid non veg food, keep mosquitos out',
                        'chicken pox': 'use neem in bathing, consume neem leaves, take vaccine, avoid public places',
                        'dengue': 'drink papaya leaf juice, avoid fatty spicy food, keep mosquitos away, keep hydrated',
                        'typhoid': 'eat high calorie vegitables, antiboitic therapy, consult doctor, medication',
                        'hepatitis a': 'Consult nearest hospital, wash hands through, avoid fatty spicy food, medication',
                        'hepatitis b': 'consult nearest hospital, vaccination, eat healthy, medication',
                        'hepatitis c': 'Consult nearest hospital, vaccination, eat healthy, medication',
                        'hepatitis d': 'consult doctor, medication, eat healthy, follow up',
                        'hepatitis e': 'stop alcohol consumption, rest, consult doctor, medication',
                        'alcoholic hepatitis': 'stop alcohol consumption, consult doctor, medication, follow up',
                        'tuberculosis': 'cover mouth, consult doctor, medication, rest',
                        'common cold': 'drink vitamin c rich drinks, take vapour, avoid cold food keep fever in check',
                        'pneumonia': 'consult doctor, medication, rest, follow up',
                        'dimorphic hemmorhoids(piles)': 'avoid fatty spicy food, consume witch hazel, warm bath with epsom salt, consume alovera juice',
                        'varicose veins': 'lie down flat and raise the leg high, use oinments, use vein compression, dont stand still for long',
                        'hypothyroidism': 'reduce stress, exercise, eat healthy, get proper sleep',
                        'hyperthyroidism': 'eat healthy, massage  use lemon balm, take radioactive iodine treatment',
                        'hypoglycemia': 'lie down on side, check in pulse, drink sugary drinks, consult doctor',
                        'osteoarthristis': 'acetaminophen, consult nearest hospital, follow up, salt baths',
                        'arthritis': 'exercise, use hot and cold therapy, try acupuncture, massage',
                        '(vertigo) paroymsal positional vertigo': 'lie down, avoid sudden change in body, avoid abrupt head movment, relax',
                        'acne': 'bath twice, avoid fatty spicy food, drink plenty of water, avoid too many products',
                        'urinary tract infection': 'drink plenty of water, increase vitamin c intake, drink cranberry juice, take probiotics',
                        'psoriasis': 'wash hands with warm soapy water, stop bleeding using pressure, consult doctor, salt baths',
                        'impetigo': 'soak affected area in warm water, use antibiotics, remove scabs with wet compressed cloth, consult doctor'}
   
   pred=tokenizer_ft.encode(text,truncation=True,padding=True, return_tensors='pt')
   op=model_ft(pred)[0]
   output=tf.argmax(op,axis=1).numpy()[0]
   print(tf.argmax(op,axis=1).numpy())
   disease=d_map[output]
   label.config(text=f"The patient could be suffering from {disease}\nSuggested precautions: {disease_precautions[disease]}")

#Create an Entry Widget
entry = Entry(win, width= 42)
entry.place(relx= .5, rely= .5, anchor= CENTER)

#Inititalize a Label widget
label= Label(win, text="Enter the Symptoms", font=('Georgia 24'),background='#000000',foreground='white')
label.place(relx=.5, rely=.3,anchor=CENTER)

#Create a Button to get the input data
button=ttk.Button(win, text= "Submit",command= get_data)
button.place(relx=.5,rely=.7,anchor=CENTER)
win.mainloop()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['itching']
itching  
[14]


: 